In [1]:
from get_training_data import *

import tensorflow as tf
import keras
import pickle
import gzip
import os
import random
import matplotlib.pyplot as plt
%matplotlib nbagg

/home/andy/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with gzip.open('train_cases.pkl.gz','rb') as fp:
    train_cases = pickle.load(fp)
type(train_cases), len(train_cases)

(tuple, 2)

In [3]:
train_X = train_cases[0]
train_y = train_cases[1]
ntrain = train_X.shape[0]
type(train_X), train_X.shape, train_X.dtype, type(train_y), train_y.shape, train_y.dtype

(numpy.ndarray,
 (16384, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (16384, 1),
 dtype('float64'))

In [4]:
with gzip.open('valid_cases.pkl.gz','rb') as fp:
    valid_cases = pickle.load(fp)
valid_X = valid_cases[0]
valid_y = valid_cases[1]
nvalid = valid_X.shape[0]
type(valid_X), valid_X.shape, valid_X.dtype, type(valid_y), valid_y.shape, valid_y.dtype

(numpy.ndarray,
 (4096, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (4096, 1),
 dtype('float64'))

In [5]:
def choose_training_case():
    i = random.randint(0, ntrain-1)
    return(train_X[i,:,:,:], train_y[i,:])

def choose_validation_case():
    i = random.randint(0, nvalid-1)
    return(valid_X[i,:,:,:], valid_y[i,:])

In [6]:
def make_batch_from_megabatch(batchsize=32, validation=False):
    get_case = choose_validation_case if validation else choose_training_case    
    xbatch = np.empty([0, INPUTS_PER_BEAT, NBEATS, NCHANNELS],dtype=np.float32)
    ybatch = np.empty([0, 1])
    for i in range(batchsize):
        case = get_case()
        x = np.expand_dims(case[0], axis=0)
        xbatch = np.concatenate([xbatch, x], axis=0)
        y = np.array(int(case[1])).reshape([1,1])
        ybatch = np.concatenate([ybatch, y], axis=0)
    return(xbatch, ybatch)

def train_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=False))

def valid_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=True))

In [7]:
# The generators are way too slow: have to fix this
for t in train_gen():
    break
t[0].shape, t[1].shape

((32, 16, 16, 128), (32, 1))

In [8]:
for v in valid_gen():
    break
v[0].shape, v[1].shape

((32, 16, 16, 128), (32, 1))

In [9]:
input_shape = (INPUTS_PER_BEAT, NBEATS, NCHANNELS)

model = keras.Sequential()
model.add(keras.layers.Conv2D(64, 1, activation='relu', input_shape=input_shape))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(32, 3, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(16, 5, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(8, 3, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(4, 1, activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(.6))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [10]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 64)        8256      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        18464     
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 16)        12816     
_________________________________________________________________
batch_normalization_3 (Batch (None, 10, 10, 16)        64        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 8)           1160      
__________

In [12]:
valid=valid_gen()
train=train_gen()
model.fit_generator(train, 
                    validation_data=valid, validation_steps=64,
                    epochs=40, 
                    steps_per_epoch=64)

Epoch 1/40
64/64 [==============================] - 2s 31ms/step - loss: 0.6924 - acc: 0.6650 - val_loss: 0.6096 - val_acc: 0.7217
Epoch 2/40
64/64 [==============================] - 1s 13ms/step - loss: 0.5976 - acc: 0.7422 - val_loss: 0.5931 - val_acc: 0.7637
Epoch 3/40
64/64 [==============================] - 1s 13ms/step - loss: 0.5551 - acc: 0.7690 - val_loss: 0.6073 - val_acc: 0.7603
Epoch 4/40
64/64 [==============================] - 1s 13ms/step - loss: 0.5482 - acc: 0.7646 - val_loss: 0.5791 - val_acc: 0.7427
Epoch 5/40
64/64 [==============================] - 1s 13ms/step - loss: 0.5536 - acc: 0.7612 - val_loss: 0.5267 - val_acc: 0.7720
Epoch 6/40
64/64 [==============================] - 1s 13ms/step - loss: 0.5399 - acc: 0.7715 - val_loss: 0.5661 - val_acc: 0.7529
Epoch 7/40
64/64 [==============================] - 1s 13ms/step - loss: 0.5419 - acc: 0.7637 - val_loss: 0.5265 - val_acc: 0.7749
Epoch 8/40
64/64 [==============================] - 1s 13ms/step - loss: 0.5283 - a

In [13]:
for i in range(10):
    song, tempo, compatible, clip = get_validation_case()
    print( song, tempo )
    c = clip_to_tf_input(resample_clip(clip))
    x = np.expand_dims(c, axis=0)
    p = model.predict(x)[0][0]
    print( 'Predicted: ', p, '   Actual: ', compatible, '\n' )

HoldMeTight 91
Predicted:  0.3630632    Actual:  False 

HangingOnTheTelephone 75
Predicted:  0.47318187    Actual:  False 

53rdAnd3rdMP 91
Predicted:  0.11714084    Actual:  False 

ChatanoogaChooChoo 41
Predicted:  0.009754806    Actual:  False 

Kalamazoo 58
Predicted:  0.18010941    Actual:  False 

BothSidesNow 45
Predicted:  0.09964603    Actual:  False 

AHardRainsAGonnaFall 43
Predicted:  0.08664252    Actual:  False 

BigJimMP 49
Predicted:  1.1158175e-05    Actual:  False 

InTheHillsOfShiloh 75
Predicted:  0.13158113    Actual:  False 

HangingOnTheTelephone 76
Predicted:  0.37688887    Actual:  False 

